In [ ]:
import tweepy
from tweepy import Stream, OAuthHandler, StreamListener
import json
import time
from collections import Counter

In [3]:
#import secret codes
from twitter_pwd import access_token, access_token_secret, consumer_key, consumer_secret

In [215]:
data_list, texts, langs, locs = [], [], [], []

class StdOutListener(StreamListener):
    """ A listener handles tweets are the received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    
    def __init__(self):
        self.data_list = []
        self.texts = []
        self.langs = []
    
    def on_data(self, data):
        jd = json.loads(data)
        self.data_list.append(jd)
        self.texts.append(jd['text'])
        self.langs.append(jd['lang'])
        try:
            print(data)
            saveFile = open('newtweets.csv', 'a')
            saveFile.write(data).encode("utf8")
            saveFile.write('/n').encode("utf8")
            saveFile.close()
            return True
        except BaseException:
            print ('failed ondata')
            time.sleep(5)

    def on_error(self, status):
        print(status)

In [216]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [103]:
#coordinates
Lviv = [23.882904,49.763526,24.163055,49.921167]
Kiev = [30.449982,50.408518,30.639496,50.495958]
Yerevan = [44.329834,40.078071,44.681396,40.296287]
Brussel = [4.258575,50.788575,4.489288,50.913424]
Barcelona = [1.835403,41.375778,2.241898,41.586688]

In [8]:
#Barcelona
l_Barc = StdOutListener()
#ASK FOR KEYWORD TO COLLECT DATA
stream_Barc = Stream(auth, l_Barc)
stream_Barc.filter(locations=Barcelona)

In [9]:
#Counter(l_Barc.langs)
# for data, lang in zip(l_Barc.data_list, l_Barc.langs):
#     print(data['user']['location'], lang)
for text in l_Barc.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [10]:
# for ee in l_Barc.data_list:
#     print(ee['place']['id'], ee['place']['place_type'])

In [101]:
l_Barc.data_list[31]['place'] #[1.835403,41.375778,2.241898,41.586688]

{'attributes': {},
 'bounding_box': {'coordinates': [[[1.360618, 41.192727],
    [1.360618, 42.323449],
    [2.77795, 42.323449],
    [2.77795, 41.192727]]],
  'type': 'Polygon'},
 'country': 'España',
 'country_code': 'ES',
 'full_name': 'Barcelona, Cataluña',
 'id': 'f3e29c4c744f3625',
 'name': 'Barcelona',
 'place_type': 'admin',
 'url': 'https://api.twitter.com/1.1/geo/id/f3e29c4c744f3625.json'}

In [11]:
# #Brussel
# l_Bru = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Bru = Stream(auth, l_Bru)
# stream_Bru.filter(locations=Brussel)

In [12]:
# #Counter(l_Bru.langs)
# for data, lang in zip(l_Bru.data_list, l_Bru.langs):
#     print(data['user']['location'], lang)

In [13]:
# #LVIV
# l_Lv = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Lv = Stream(auth, l_Lv)
# stream_Lv.filter(locations=Lviv)

In [14]:
# #YEREVAN
# l_Yer = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Yer = Stream(auth, l_Yer)
# stream_Yer.filter(locations=Yerevan)

In [15]:
# for data, lang in zip(l_Yer.data_list, l_Yer.langs):
#     print(data['user']['location'], lang)

In [16]:
# for text in l_Yer.texts:
#     if type(text) == str:
#         print(text)
#     else:
#         print(text.decode('utf-8'))

In [17]:
# for data, lang in zip(data_list, langs):
#     print(data['user']['location'], lang)

In [66]:
from collections import Counter
Counter(langs)

Counter({'en': 8, 'nl': 1, 'pl': 1, 'ru': 54, 'tl': 1, 'uk': 8, 'und': 6})

In [18]:
#KIEV
l_Kiev = StdOutListener()
#ASK FOR KEYWORD TO COLLECT DATA
stream_Kiev = Stream(auth, l_Kiev)
stream_Kiev.filter(locations=Kiev)

In [19]:
#Counter(l_Kiev.langs)
# for data, lang in zip(l_Kiev.data_list, l_Kiev.langs):
#     print(data['user']['location'], lang)
for text in l_Kiev.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [20]:
for data in l_Kiev.data_list:
    print(data['place']['id'], data['place']['place_type'], 
          data['place']['country'], data['user']['location'])

In [62]:
for text in texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))


включила.. переключила станцию... снова переключила...  SORRY BAE  #BTSBBMAs
Когда попросили улыбнуться 🙃

Vote for #BTSBBMAs #BBMAs https://t.co/v54cpHqxto
@yuukierosrose но как же платюшко D:
#btsbbmas шестьдесят пятый Джин
@veraizmaylova41 прикольно но по Турецки среднее слово. это характер)))
💜 #BeYourWINGS #BTSBBMAs невозможно описать как-либо то, как изменили мою жизнь эти парни, как я ожила, увидела и мир и как я благодарна 💜
I love you so much thank you so much #BTSBBMAs
@wulfhere2000 @poroshenko ти ж бажав 320 дівчат в читачах? 🤣
#BeYourWINGS 💜   #BTSBBMAs https://t.co/U2DHYGGzB4
То є Львів старенький, мудрий, сивий дядько.✨
Кожному залишить кавалочок на згадку.👌🏻😍 https://t.co/A8oPB81bou
#BTSBBMAs @BTS_twt
Чи може бути цей дощ.....слізьми Хобі?
@inart66 дык страждущим и нужно
В Одессе водителя «Яндекс.Такси» заподозрили в убийстве студентки Одесской юридической академии Татьяны Маснюк… https://t.co/Gxq3pgBccx
#btsbbmas шестьдесят шестой Джин
Хана туль сэт #BTSBBMAs
пэндехо по

In [ ]:
list_ukr_cities = ['Херсон', 'Киев', ]

In [ ]:
# https://twitter.com/search?f=users&q=Дніпро&src=typd

In [272]:
## USERS, FOLLOWERS, TIMELINES, LANGS

In [252]:
import tweepy
import time


def get_twitter_followers(account_name, max_num=100):
    list_followers=[]
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    users = tweepy.Cursor(api.followers, screen_name=account_name).items(max_num)
    #i = 0
    while True:
        try:
            #print(i)
            user = next(users)
            list_followers.append(user)
            #i+=1
        except tweepy.TweepError as e:
            if 'Read timed out' in str(e):
                print('fall here')
                print(e)
                time.sleep(5)
            else:
                time.sleep(60*16)
                user = next(users)
        except StopIteration:
            break
        #print ("@" + user.screen_name)
    return list_followers
        
        
def get_user_timeline(account_name, max_num=10):
    list_tweets=[]
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    timeline = tweepy.Cursor(api.user_timeline, screen_name=account_name).items(max_num)
    i=0
    while True:
        try:
            tw = next(timeline)
            list_tweets.append(tw)
        except tweepy.TweepError as e:
            if '401' in str(e):    
                print(e)
                time.sleep(10)
                break
            else:
                time.sleep(60*15)
                tw = next(timeline)
        except StopIteration:
            break
    return list_tweets

In [253]:
def get_lang_twts_followers(list_followers, max_num_followers=None):
    text_tl_followers=[]
    lang_tl_followers=[]
    if max_num_followers:
        list_followers = list_followers[:max_num_followers]
    for idx, f in enumerate(list_followers):
        #print(idx)
        tl = get_user_timeline(f.screen_name, max_num=100)
        text_tl_followers.extend([tw.text for tw in tl])
        lang_tl_followers.extend([tw.lang for tw in tl])
    return text_tl_followers, lang_tl_followers

In [26]:
hr_lviv_followers = get_twitter_followers('hromadskelviv')

In [27]:
Counter([f.lang for f in hr_lviv_followers])

Counter({'en': 8, 'es': 1, 'pl': 2, 'ru': 38, 'uk': 51})

In [260]:
# kiev polytech univ
knu_followers = get_twitter_followers('KyivUniversity', max_num=250)

In [261]:
Counter([f.lang for f in knu_followers])

Counter({'ar': 2, 'en': 43, 'en-gb': 2, 'fr': 1, 'ru': 129, 'tr': 2, 'uk': 71})

In [262]:
# ukrainska pravda
ukr_pravda_followers = get_twitter_followers('ukrpravda_news', max_num=250)

Rate limit reached. Sleeping for: 136


KeyboardInterrupt: 

In [21]:
#klitschko
vk_followers = get_twitter_followers('Vitaliy_Klychko', max_num=250)

In [251]:
Counter([f.lang for f in vk_followers])

Counter({'en': 12, 'es': 1, 'it': 1, 'pl': 1, 'ru': 143, 'uk': 92})

In [255]:
vk_texts, vk_langs = get_lang_twts_followers(vk_followers, max_num_followers=100)

Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401


In [256]:
Counter(vk_langs)

Counter({'ar': 1,
         'bg': 4,
         'cs': 7,
         'en': 123,
         'es': 142,
         'fr': 1,
         'hi': 85,
         'ne': 1,
         'no': 1,
         'pl': 68,
         'ru': 413,
         'sl': 1,
         'uk': 123,
         'und': 202})

In [22]:
ak_followers = get_twitter_followers('auto_kiev', max_num=600)

In [229]:
Counter([f.lang for f in ak_followers])

Counter({'de': 1,
         'en': 41,
         'es': 1,
         'fr': 1,
         'pl': 1,
         'ru': 464,
         'uk': 90,
         'zh-cn': 1})

In [40]:
ak_twts = get_user_timeline('auto_kiev', max_num=100)

In [42]:
Counter([tw.lang for tw in ak_twts])

Counter({'ru': 73, 'uk': 26, 'und': 1})

In [233]:
txts_ak, langs_ak = get_lang_twts_followers(ak_followers)

Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Rate limit reached. Sleeping for: 282
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 

In [248]:
und_txt = [txt for txt,lang in zip(txts_ak, langs_ak) if lang == 'it']
und_txt

In [234]:
# lang_tl_followers=[]
# for idx, f in enumerate(ak_followers[:100]):
#     print(idx)
#     tl = get_user_timeline(f.screen_name, max_num=100)
#     lang_tl_followers.extend([tw.lang for tw in tl])

In [235]:
counts=Counter(langs_ak)
counts['uk']/(counts['uk'] + counts['ru'])

0.20796656323222087

In [236]:
counts

Counter({'ar': 9,
         'bg': 171,
         'cs': 122,
         'cy': 3,
         'da': 10,
         'de': 32,
         'en': 1748,
         'es': 108,
         'et': 6,
         'eu': 4,
         'fi': 1,
         'fr': 78,
         'ht': 4,
         'hu': 5,
         'in': 28,
         'is': 2,
         'it': 12,
         'iw': 1,
         'ja': 3,
         'ka': 16,
         'ko': 2,
         'lt': 3,
         'lv': 1,
         'nl': 4,
         'no': 4,
         'pl': 20,
         'pt': 22,
         'ro': 16,
         'ru': 12507,
         'sl': 1,
         'sr': 18,
         'sv': 1,
         'tl': 6,
         'tr': 4,
         'uk': 3284,
         'und': 3273,
         'zh': 1})

In [263]:
import deepdish as dd

In [264]:
kiev_lang_data = {'vitaly_klitschko':
                  {'followers':vk_followers, 'twts_followers':vk_texts,'lang_twts_followers':vk_langs},
                  'avto_kiev':
                  {'followers':ak_followers, 'twts_followers':txts_ak, 'lang_twts_followers':langs_ak}, 
                  'ukrainska_pravda':{}}

In [266]:
dd.io.save('kiev_twt_data.h5', kiev_lang_data)

### CATALUNYA

In [21]:
# Catalunya
list_acc = ['bcn_ajuntament', 'girona_cat']

'ca'

In [143]:
aj_bcn_followers = get_twitter_followers('bcn_ajuntament', max_num=200)

In [33]:
Counter([f.lang for f in aj_bcn_followers])

Counter({'ca': 18,
         'de': 1,
         'en': 15,
         'en-gb': 3,
         'es': 153,
         'fr': 4,
         'ja': 1,
         'sv': 2,
         'zh-CN': 1,
         'zh-cn': 1,
         'zh-tw': 1})

In [144]:
aj_bcn_followers[36].screen_name

'Leetiimooldeer4'

In [23]:
txts, langs = get_lang_twts_followers(aj_bcn_followers)

In [146]:
Counter(langs)

Counter({'ar': 3,
         'de': 2,
         'en': 432,
         'es': 1046,
         'et': 1,
         'eu': 7,
         'fi': 1,
         'fr': 118,
         'ht': 3,
         'in': 6,
         'it': 70,
         'ja': 3,
         'nl': 1,
         'pt': 21,
         'ro': 2,
         'sv': 1,
         'tl': 2,
         'tr': 2,
         'und': 868,
         'zh': 1})

In [25]:
[(tw, lang) for tw, lang in zip(txts,langs) if lang == 'und']

In [157]:
from langdetect import detect

In [167]:
import re

In [169]:
pattern = re.compile(r"\w+")
re.findall(pattern, "https://t.co/UShzr7uCYU")

['https', 't', 'co', 'UShzr7uCYU']

In [172]:
#langs_detected = [detect(txt) for txt in txts if len(txts) > 10]
langs_detected=[]
for i,txt in enumerate(txts):
    #print(i, txt)
    try:
        langs_detected.append(detect(txt))
    except:
        continue

In [173]:
Counter(langs_detected)

Counter({'af': 5,
         'ar': 3,
         'ca': 730,
         'cs': 2,
         'cy': 2,
         'da': 3,
         'de': 15,
         'en': 435,
         'es': 970,
         'et': 6,
         'fi': 4,
         'fr': 119,
         'hr': 3,
         'hu': 1,
         'id': 8,
         'it': 93,
         'ja': 3,
         'lt': 4,
         'nl': 4,
         'no': 4,
         'pt': 55,
         'ro': 8,
         'sk': 2,
         'sl': 1,
         'so': 3,
         'sv': 3,
         'sw': 2,
         'tl': 5,
         'tr': 3})

In [46]:
aj_gr_followers = get_twitter_followers('girona_cat', max_num=200)

In [48]:
aj_gr_twts = get_user_timeline('girona_cat', max_num=100)

In [47]:
Counter([f.lang for f in aj_gr_followers])

Counter({'ca': 76, 'de': 1, 'en': 26, 'es': 91, 'fr': 3, 'ja': 1, 'pt': 2})

In [56]:
aj_gr_twts[2].id

866377201733775360

In [ ]:
ap

In [163]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [131]:
url = "http://www.worldfootball.net/teams/dinamo-kiev/2017/2/"
html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

In [189]:
dynamo_players = pd.read_html(url, encoding='utf8')[1][2].dropna().values

In [190]:
dynamo_players

array(['Maksim Koval', 'Artur Rudko', 'Antunes', 'Mykyta Burda',
       'Tamás Kádár', 'Evgen Khacheridi', 'Bogdan Mykhaylychenko',
       'Zurab Ochihava', 'Aleksandar Pantić', 'Oleksandr Tymchyk',
       'Domagoj Vida', 'Vitaliy Buyalskiy', 'Valeriy Fedorchuk',
       'Denys Garmash', 'Nikita Korzun', 'Mykyta Kravchenko',
       'Mikola Morozyuk', 'Pavel Orekhovskiy', 'Serhiy Rybalka',
       'Volodymyr Shepeliev', 'Sergiy Sydorchuk', 'Viktor Tsygankov',
       'Artem Besedin', 'Derlis González', 'Roman Yaremchuk',
       'Andriy Yarmolenko', 'Sergiy Rebrov'], dtype=object)

In [193]:
dyn_play_countr = pd.read_html(url, encoding='utf8')[1][[2,4]].dropna().values

In [194]:
dyn_play_countr

array([['Maksim Koval', 'Ukraine'],
       ['Artur Rudko', 'Ukraine'],
       ['Antunes', 'Portugal'],
       ['Mykyta Burda', 'Ukraine'],
       ['Tamás Kádár', 'Hungary'],
       ['Evgen Khacheridi', 'Ukraine'],
       ['Bogdan Mykhaylychenko', 'Ukraine'],
       ['Zurab Ochihava', 'Ukraine'],
       ['Aleksandar Pantić', 'Serbia'],
       ['Oleksandr Tymchyk', 'Ukraine'],
       ['Domagoj Vida', 'Croatia'],
       ['Vitaliy Buyalskiy', 'Ukraine'],
       ['Valeriy Fedorchuk', 'Ukraine'],
       ['Denys Garmash', 'Ukraine'],
       ['Nikita Korzun', 'Belarus'],
       ['Mykyta Kravchenko', 'Ukraine'],
       ['Mikola Morozyuk', 'Ukraine'],
       ['Pavel Orekhovskiy', 'Ukraine'],
       ['Serhiy Rybalka', 'Ukraine'],
       ['Volodymyr Shepeliev', 'Ukraine'],
       ['Sergiy Sydorchuk', 'Ukraine'],
       ['Viktor Tsygankov', 'Ukraine'],
       ['Artem Besedin', 'Ukraine'],
       ['Derlis González', 'Paraguay'],
       ['Roman Yaremchuk', 'Ukraine'],
       ['Andriy Yarmolenko', 'Uk

In [175]:
from langdetect import detect_langs